In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
from tqdm.notebook import tqdm
import re
import time
from functools import partial
import csv

In [2]:
list_topic=['100','101','102','103','104','105']
list_subtopic=[['264','265','268','266','267','269'],#청와대,국회/정당,북한,행정,국방/외교,정치일반
               ['259','258','261','771','260','262','310','263'],#금융,증권,산업/재계,중기/벤처,부동산,글로벌 경제,생활경제,경제 일반
               ['249','250','251','254','252','59b','255','256','257'],#사건사고,교육,노동,언론,환경,인권/복지,식품/의료,지역,사회일반
               ['241','239','240','237','238','376','245'],#건강정보,자동차/시승기,도로/교통,여행/레저,음식/맛집,패션/뷰티,생활문화 일반
               ['231','232','233','234','322'],#아시아/호주,미국/중남미,유럽,중동/아프리카,세계 일반
               ['731','226','227','230','732','283','229','228'],#모바일,인터넷/sns,통신/뉴미디어,IT일반,보안/해킹,컴퓨터,게임/리뷰,과학일반
              ]
topic_dict={'100':"정치",'101':"경제",'102':"사회",'103':"생활/문화",'104':"국제",'105':"IT/과학"}
topic_sub={'264':"청와대",'265':'국회/정당','268':'북한','266':'행정','267':'국방/외교','269':'정치일반',
           '259':'금융','258':'증권','261':'산업/재계','771':'중기/벤처','260':'부동산','262':'글로벌 경제','310':'생활경제','263':'경제일반',
           '249':'사건사고','250':'교육','251':'노동','254':'언론','252':'환경','59b':'인권/복지','255':'식품/의료','256':'지역','257':'사회일반',
           '241':'건강정보','239':'자동차/시승기','240':'도로/교통','237':'여행/레저','238':'음식/맛집','376':'패션/뷰티','245':'생활문화 일반',
           '231':'아시아/호주','232':'미국/중남미','233':'유럽','234':'중동/아프리카','322':'세계 일반',
           '731':'모바일','226':'인터넷/sns','227':'통신/뉴미디어','230':'IT일반','732':'보안/해킹','283':'컴퓨터','229':'게임/리뷰','228':'과학일반'}

In [3]:
import pandas as pd
import numpy as np
#기간설정
dt_index = pd.date_range(start='20200101', end='20201229')

In [4]:
headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15',}

In [5]:
def no_space(text):
    text1=re.sub('&nbsp; | &nbsp;| \n|\t|\r','',text)
    text2=re.sub('\n\n','',text1)
    return text2

In [6]:

#페이지 수 세는 def
def paging(page,page_list):
    
    for p in page:
        if(p.text=="이전"):
            pass
        elif(p.text=="다음"):
            page_list.append("https://news.naver.com/main/read.nhn"+p['href'])
            html = "https://news.naver.com/main/list.nhn"+p['href']
            url=requests.get(html,headers=headers)
            bsObject = BeautifulSoup(url.content, "html.parser")
            page=bsObject.select("#main_content > div.paging > a")
            paging(page,page_list)
        else:
            page_list.append("https://news.naver.com/main/read.nhn"+p['href'])



In [7]:
def filt(k):
    k.replace('\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}','')
    try:
        text=k.split("▶")[0].replace('\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}','')
    except:
        pass
    return text

In [8]:
f = open('naver.csv','w', newline='')
wr = csv.writer(f)
wr.writerow(['date','title','topic','sub_topic','text'])
 
f.close()

In [9]:
#크롤링 def
def get_news(date,topic,sub_topic,main_url):

        
        

    html = main_url
    html=requests.get(html,headers=headers)
    bsObject = BeautifulSoup(html.content, "html.parser")

    for meta in bsObject.select("#main_content > div.list_body.newsflash_body > ul > li > dl "):
        meta=meta.select_one("a")
        link=meta['href']
        date=date
        url=link
        url=requests.get(url,headers=headers)
        bs_obj=BeautifulSoup(url.content, "html.parser")  
        try:
            title=bs_obj.select_one("#articleTitle").text         
            x=bs_obj.select_one("#articleBodyContents").text
            title=no_space(title)
            text=no_space(x)
            text=filt(text)
            df=pd.DataFrame(columns=["date", "title", "topic","sub_topic","text"])
            df=df.append(pd.Series([date,title,topic_dict[topic],topic_sub[sub_topic],text],index=df.columns),ignore_index=True)
            df.to_csv('naver.csv', mode='a', header=False, index=False)
            print("date: "+date)
            print("title: "+title)
            print("topic: "+topic_dict[topic])
            print("sub_topic: "+topic_sub[sub_topic])
            print("address: "+link)
        except:
            print("-----------------error----------------address: "+link)



In [10]:
#크롤링 시작
num=0
start_time=time.time()
for num_main in tqdm(range(len(list_topic)),desc='Topic_Progress',leave=False):
    for num_sub in tqdm(range(len(list_subtopic[num])),desc='Sub_Topic_Progress',leave=False):
        main_url="https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2="+list_subtopic[num_main][num_sub]+"&sid1="+list_topic[num_main]+"&date="
        for num_date in tqdm(range(len(dt_index.date)),desc='Day_Progress'):
            date=str(dt_index.date[num_date])
            date=date.split('-')[0]+date.split('-')[1]+date.split('-')[2]
            page_list=[]
            get_news(date,list_topic[num_main],list_subtopic[num_main][num_sub],main_url+date)    
            
            
            html = main_url+date
            html=requests.get(html,headers=headers)
            bsObject = BeautifulSoup(html.content, "html.parser")     
            
            page=bsObject.select("#main_content > div.paging > a")
            paging(page,page_list)
            
            for k in page_list:
                get_news(date,list_topic[num_main],list_subtopic[num_main][num_sub],k)




            
            time.sleep(0.05)
            
            
    num+=1
print(time.time()-start_time)

date: 20200101
title: 문 대통령, 해맞이 산행 [포토뉴스]
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=032&aid=0002983868
date: 20200101
title: 靑, 이르면 6일 비서진 교체·조직 개편…윤건영 구로을 출마 유력
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004387220
date: 20200101
title: 추미애, 파격인사로 윤석열 손발 묶는다
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=011&aid=0003674411
date: 20200101
title: 신년사도, 남한 언급도 없는 김정은…靑 말 아낀 채 의중 파악(종합)
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004387208
date: 20200101
title: 文대통령, 6일께 비서관 인사…윤건영 총선출마 가닥
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=008&aid=0004333375
date: 20200101
title: "열심히 일한 만큼 새해 행복할 자격 있어"
topic: 정치
sub_topic: 청와대
address: https://new

date: 20200103
title: "한미일 안보 담당 고위급 회의 이르면 8일 개최 조율"<교도>
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011314178
date: 20200103
title: 靑참모진, 4월 총선 출마 여부 놓고 설왕설래 만발
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=018&aid=0004548622
date: 20200103
title: 주형철·고민정…靑참모, 공직자 사퇴시한 임박에 출마 막판 고심
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004390996
date: 20200103
title: 文, 올해 1호 수출용 친환경차 타고 "대한민국, 달리자"
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=079&aid=0003308071
date: 20200103
title: 文대통령, 전국 119상황실에 靑 텃밭서 수확한 작물茶 선물
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011314134
date: 20200103
title: 靑, 이르면 6일 윤건영·주형철 등 총선 출마자 교체할 듯
topic: 정치
sub_topi

date: 20200105
title: 文대통령, 7일 공식 신년사 통해 검찰개혁·경제혁신 로드맵 제시
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=018&aid=0004549125
date: 20200105
title: 文대통령, 이르면 이번주 '靑 개편' 단행
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=014&aid=0004352776
date: 20200105
title: 文, 7일 신년사···경제 '확실한 성과' 강조할 듯
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=011&aid=0003675656
date: 20200105
title: 문 대통령, 7일 신년사·국무회의로 후반기 국정운영 기조 제시...인사 및 조직개편에도 관심
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=030&aid=0002860733
date: 20200105
title: [주간 문재인] ‘발등의 불’된 수출…文대통령, 연초 평택으로 달려간 이유?
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=018&aid=0004549081
date: 20200105
title: 미리 보는 文대통령 신년사…'상생·도약' 메시지에 방점
to

date: 20200107
title: 민주노총, 文대통령 신년사에 "후퇴한 노동정책 반성 없어"
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=001&aid=0011320725
date: 20200107
title: 수출 두자릿수 내려앉았는데… 지지층 겨냥 `셀프 칭찬`
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=029&aid=0002579485
date: 20200108
title: 검찰 요직 새로 꿰찬 인사들 들여다 보니···
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=011&aid=0003677820
date: 20200108
title: 문화예술인 신년인사회 인사말하는 문재인 대통령
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004399272
date: 20200108
title: 문화예술인 신년인사회 참석한 문재인 대통령
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004399271
date: 20200108
title: 조정래 작가 인사말 경청하는 문재인 대통령과 김정숙 여사
topic: 정치
sub_topic: 청와대
address: https://news

date: 20200110
title: 靑 "보여주기식 수사" 강력 비난…檢 "영장에 근거" 즉각 반박
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=009&aid=0004498560
date: 20200110
title: 檢 "청와대, 자료제출 거부해 압수수색"…빈손 철수
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=421&aid=0004403219
date: 20200110
title: 검찰, 靑압수수색 중단…"영장제시 했으나 자료 못받아"
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=003&aid=0009649185
date: 20200110
title: 정의용 "트럼프, 호르무즈 파병 언급 안해"
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=014&aid=0004355968
date: 20200110
title: 靑 "檢, 압수 대상 특정 안된 보여주기식 수사... 강한 유감"
topic: 정치
sub_topic: 청와대
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=264&oid=014&aid=0004355966
date: 20200110
title: 특정 안 된 압수 영장에 靑 압수수색 무산…"보여주기 수사"(종합)
topic: 정치
sub_topic: 청

...

date: 20200701
title: 민주, ‘일하는 국회법’ 1호 법안 추진
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=020&aid=0003295352
date: 20200701
title: 예결소위, 3차 추경 감액 심사 돌입…野 불참 속 일사천리(종합)
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=003&aid=0009945687
date: 20200701
title: 김부겸, 다음주 '책임론' 강조하며 당권 도전
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=469&aid=0000511578
date: 20200701
title: 한상혁 방통위원장 인사청문안 제출…재산 10억3000만원
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=003&aid=0009945677
date: 20200701
title: 여당 ‘윤석열 성토대회’…추 법무, 특임검사 투입엔 ‘선긋기’
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=032&aid=0003018217
date: 20200701
title: 예결소위, 추경 감액 與 단독 심사...‘일사천리’
topic: 정치
sub_topic: 국회/정당

date: 20200703
title: 정부측 인사하는 정세균 국무총리
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=003&aid=0009949903
date: 20200703
title: 정부측 인사하는 정세균 국무총리
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=003&aid=0009949902
date: 20200703
title: 의사진행발언하는 박홍근 더불어민주당 예결위 간사
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=003&aid=0009949901
date: 20200704
title: [국정농담] 총리도 아니라는데, 김현미는 왜 집값에 당당할까
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=011&aid=0003763417
date: 20200704
title: 조국, 반기 든 검사장들 맹비난…7년 전엔 '의무'라더니
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=015&aid=0004373766
date: 20200704
title: 진중권, 北 최선희 담화에 "트럼프 재선 불투명…현실감은 안 잃었네"
topic: 정치
sub_topic: 국회/정당
address: http

date: 20200706
title: 안희정 모친상에 여권 조문행렬…정의당 "성폭력에도 지지않는 정치권 연대"(종합)
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=421&aid=0004738553
date: 20200706
title: 김태년, 국회의원 후보 '1주택 서약' 이행 앞당길 것
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=015&aid=0004374960
date: 20200706
title: 與 '1주택 서약' 이행 앞당긴다…김태년 "단축방안 마련"
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=001&aid=0011727983
date: 20200706
title: YTN 변상욱, 최강욱 불러놓고 "대검 검사장들 의견 공개 어떻게 생각?"
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=015&aid=0004374958
date: 20200706
title: 김진표 의원 ‘군공항 이전 및 지원 특별법 개정안’ 발의
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=022&aid=0003481722
date: 20200706
title: 이재명 "소주성은 적확한 경제해법, 괴물은 국

date: 20200708
title: 은성수도 '똘똘한 한 채' 남겼다..서초 두고 세종 팔아
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=018&aid=0004684491
date: 20200708
title: 이낙연 “집값 급등에 깊은 책임 느껴… 국민께 송구”
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=469&aid=0000513888
date: 20200708
title: 이낙연 "부동산 폭등 송구…서울 주택규제 완화 검토"(종합)
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=003&aid=0009957665
date: 20200708
title: 이낙연 "부동산 문제, 깊은 책임 느끼고 국민께 송구"(종합)
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=001&aid=0011733626
date: 20200708
title: 박 의장, 월세라던 대전 집 아들에 증여하고 관리비 냈다
topic: 정치
sub_topic: 국회/정당
address: https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=100&sid2=265&oid=018&aid=0004684490
date: 20200708
title: 이낙연 "부동산 문제, 저도 깊은 책임 느껴"(종합2보)
topic: 정치
sub

KeyboardInterrupt: 

In [11]:
df=pd.read_csv("naver.csv")

In [12]:
df

,date,title,topic,sub_topic,text
0,20200101,"문 대통령, 해맞이 산행 [포토뉴스]",정치,청와대,\n문재인 대통령이 1일 ‘2019년을 빛낸 의인’들과 함께 신년 해맞이를 위해 서...
1,20200101,"靑, 이르면 6일 비서진 교체·조직 개편…윤건영 구로을 출마 유력",정치,청와대,\n윤 후임에 오종식·신상엽·이진석 등 거론고민정 불출마로 입장 정리한듯…사실상 중...
2,20200101,"추미애, 파격인사로 윤석열 손발 묶는다",정치,청와대,"\n청와대, 경찰에 검사장 승진 대상자 세평 취합 주문추미애 법무부 장관 후보자가 ..."
3,20200101,"신년사도, 남한 언급도 없는 김정은…靑 말 아낀 채 의중 파악(종합)",정치,청와대,"\n국가안보실 중심으로 신년사 대기…낮 12시30분 넘자 '안한다' 결론金 ""곧 새..."
4,20200101,"文대통령, 6일께 비서관 인사…윤건영 총선출마 가닥",정치,청와대,"[머니투데이 김성휘 기자] [[the300]국정상황실 포함 조직개편, '4월 총선..."
...,...,...,...,...,...
11085,20200709,"민주당, 부동산 대책 당정 회의 취소…박 시장 실종 의식한 듯",정치,국회/정당,"\n김태년 더불어민주당 원내대표가 8일 국회에서 열린 최고위원회의에 참석, 모두발언..."
11086,20200709,"윤건영 ""北 김정은 위원장도 10월 3차 북미정상회담 생각할 것""",정치,국회/정당,\n윤건영 더불어민주당 원이 지난달 16일 국회 의원회관에서 열린 포스트 코로나와 ...
11087,20200709,[영상]故최숙현 사건 토론회서 피겨 꿈나무 엄마의 눈물,정치,국회/정당,동영상 뉴스(서울=뉴스1) 송영성 기자 = 고 최숙현 선수 사건으로 스포츠계에 만연...
11088,20200709,"與, 박원순 실종에 내일 부동산 당정협의 전격 취소",정치,국회/정당,"\n민주당 지도부, 긴급 대책회의 열어 취소 결정오전 중 정부 부동산 종합대책 발표..."
